In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector_repo_id = "jesmine0820/fake_review_detection_model"
fake_repo_id = "jesmine0820/fake_review_clustering_model"

c:\Users\User\anaconda3\envs\environment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_model_and_tokenizer(model_dir):
    tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}/tokenizer")
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.to(device)
    model.eval()
    return tokenizer, model

def predict_review(texts, tokenizer, model):
    if isinstance(texts, str):
        texts = [texts]

    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    return preds.cpu().tolist(), probs.cpu().tolist()

examples = [
    "This product is amazing! Highly recommended.",
    "Awful! Completely broken after one use. Do not buy.",
    "I will never stay at this hotel again.",
    "Room was clean, staff were friendly and breakfast was great."
]

model_paths = [
    './fake_review_detection_model/bert-base-uncased'
]

'''
,
    './fake_review_detection_model/roberta-base',
    './fake_review_detection_model/google/electra-base-discriminator'
'''

results = {'text': examples}

for i, path in enumerate(model_paths, 1):
    tokenizer, model = load_model_and_tokenizer(path)
    labels, _ = predict_review(examples, tokenizer, model)
    results[f'model{i}'] = labels  

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './fake_review_detection_model/bert-base-uncased/tokenizer'. Use `repo_type` argument if needed.

In [ ]:
df = pd.DataFrame(results)
print(df)